In [23]:
import bokeh
from bokeh.plotting import show, save
import numpy as np
import re
from pathlib import Path

import flowkit as fk

bokeh.io.output_notebook()

Loading BokehJS ...

In [24]:
import pandas as pd
import polars as pl
import polars.selectors as cs
from polars import col, lit, when

In [25]:
batch='20250908'

# process df

In [26]:
df = pl.read_csv('eg_data/sample_info.csv').fill_null('')

In [27]:
df.group_by('cell').len()

cell,len
str,u32
"""PETcon-antigen-display-myc""",5


In [28]:
df = df.with_columns(
    sample_id = pl.format('{}:{}\n{}', col("sample"), col("ab"), col("cell"))
)

In [29]:
def gen_path(row):
    root = f'./eg_data'
    plate = row["plate"].lower()
    pos = row["pos"].lower()
    pos_row = pos[0]
    pos_col0 = pos[1:].rjust(2, '0').lower()

    return f'{root}/{plate}/{batch}-{plate}_{pos}_{pos_row}{pos_col0}.fcs'

In [30]:
paths = df.to_pandas().apply(gen_path, axis=1)

In [31]:
df = df.with_columns(path=pl.Series(paths))

# parse

In [34]:
session = fk.Session()

In [35]:
error_samples = []
for sample_id, path in zip(df['sample_id'], df['path']):
    if Path(path).exists():
        try:
            session.add_samples(fk.Sample(path, sample_id=sample_id))
        except:
            print(f'Parse Error: {sample_id}, {path}')
            error_samples.append(sample_id)
    else:
        print(f'Not found: {path}')

In [36]:
# remove error sample
df = df.filter(~col("sample_id").is_in(error_samples))

In [37]:
session.add_transform(fk.transforms.AsinhTransform('ash', param_t=262144, param_m=5.42, param_a=0.5))

In [38]:
gate_r1 = fk.gates.RectangleGate('R1', dimensions=[
        fk.Dimension('FSC-A', range_min=50000, range_max=200000), 
        fk.Dimension('SSC-A', range_min=40000, range_max=180000)
    ])
gate_r2 = fk.gates.RectangleGate('R2', dimensions=[
        fk.Dimension('FITC-A', range_min=0, range_max=1, transformation_ref='ash'), # myc
        fk.Dimension('PE-A', range_min=0, range_max=1, transformation_ref='ash') # ab
    ])

gate_r3 = fk.gates.RectangleGate('R3', dimensions=[
        fk.Dimension('FITC-A', range_min=0, range_max=1, transformation_ref='ash'), # myc
        fk.Dimension('APC-A', range_min=0, range_max=1, transformation_ref='ash') # ha
    ])

In [39]:
div_value_h = 0.55
div_value_v = 0.5

div_h = fk.QuadrantDivider('div-h', 'FITC-A', values=[div_value_h],
    compensation_ref='uncompensated', transformation_ref='ash')
div_v = fk.QuadrantDivider('div-v', 'PE-A', values=[div_value_v],
    compensation_ref='uncompensated', transformation_ref='ash')

quad_topright = fk.gates.Quadrant(
    quadrant_id='topright',
    divider_refs=['div-h', 'div-v'],
    divider_ranges=[(div_value_h, None), (div_value_v, None)]
)
quad_downright = fk.gates.Quadrant(
    quadrant_id='downright',
    divider_refs=['div-h', 'div-v'],
    divider_ranges=[(div_value_h, None), (None, div_value_v)]
)
quad_topleft = fk.gates.Quadrant(
    quadrant_id='topleft',
    divider_refs=['div-h', 'div-v'],
    divider_ranges=[(None, div_value_h), (div_value_v, None)]
)
quad_downleft = fk.gates.Quadrant(
    quadrant_id='downleft',
    divider_refs=['div-h', 'div-v'],
    divider_ranges=[(None, div_value_h), (None, div_value_v)]
)

quad_gate = fk.gates.QuadrantGate(
    'quadgate',
    dividers=[div_h, div_v],
    quadrants=[quad_topright, quad_downright, quad_topleft, quad_downleft]
)

In [40]:
div_value_h = 0.55
div_value_v = 0.5

div_h = fk.QuadrantDivider('div-h-ext', 'FITC-A', values=[div_value_h],
    compensation_ref='uncompensated', transformation_ref='ash')
div_v = fk.QuadrantDivider('div-v-ext', 'APC-A', values=[div_value_v],
    compensation_ref='uncompensated', transformation_ref='ash')

quad_topright = fk.gates.Quadrant(
    quadrant_id='topright-ext',
    divider_refs=['div-h-ext', 'div-v-ext'],
    divider_ranges=[(div_value_h, None), (div_value_v, None)]
)
quad_downright = fk.gates.Quadrant(
    quadrant_id='downright-ext',
    divider_refs=['div-h-ext', 'div-v-ext'],
    divider_ranges=[(div_value_h, None), (None, div_value_v)]
)
quad_topleft = fk.gates.Quadrant(
    quadrant_id='topleft-ext',
    divider_refs=['div-h-ext', 'div-v-ext'],
    divider_ranges=[(None, div_value_h), (div_value_v, None)]
)

quad_downleft = fk.gates.Quadrant(
    quadrant_id='downleft-ext',
    divider_refs=['div-h-ext', 'div-v-ext'],
    divider_ranges=[(None, div_value_h), (None, div_value_v)]
)

quad_gate_ha_myc = fk.gates.QuadrantGate(
    'quadgate_ha_myc',
    dividers=[div_h, div_v],
    quadrants=[quad_topright, quad_downright, quad_topleft, quad_downleft]
)

In [41]:
session.add_gate(gate_r1, gate_path=('root',))
session.add_gate(gate_r2, gate_path=('root', 'R1'))
session.add_gate(gate_r3, gate_path=('root', 'R1'))
session.add_gate(quad_gate, gate_path=('root', 'R1', 'R2'))
session.add_gate(quad_gate_ha_myc, gate_path=('root', 'R1', 'R3'))

In [42]:
session.analyze_samples()

In [43]:
print(session.gating_strategy.get_gate_hierarchy(output='ascii'))

root
╰── R1
    ├── R2
    │   ╰── quadgate
    │       ├── topright
    │       ├── downright
    │       ├── topleft
    │       ╰── downleft
    ╰── R3
        ╰── quadgate_ha_myc
            ├── topright-ext
            ├── downright-ext
            ├── topleft-ext
            ╰── downleft-ext


In [44]:
session.get_gating_results(session.get_sample_ids()[0]).report

,sample,gate_path,gate_name,gate_type,quadrant_parent,parent,count,absolute_percent,relative_percent,level
0,S1:/\nPETcon-antigen-display-myc,"(root,)",R1,RectangleGate,None,root,36518,73.036,73.036000,1
1,S1:/\nPETcon-antigen-display-myc,"(root, R1)",R2,RectangleGate,None,R1,24383,48.766,66.769812,2
2,S1:/\nPETcon-antigen-display-myc,"(root, R1)",R3,RectangleGate,None,R1,22716,45.432,62.204940,2
6,S1:/\nPETcon-antigen-display-myc,"(root, R1, R2)",downleft,QuadrantGate,quadgate,R2,24298,48.596,99.651396,3
10,S1:/\nPETcon-antigen-display-myc,"(root, R1, R3)",downleft-ext,QuadrantGate,quadgate_ha_myc,R3,22593,45.186,99.458531,3
4,S1:/\nPETcon-antigen-display-myc,"(root, R1, R2)",downright,QuadrantGate,quadgate,R2,64,0.128,0.262478,3
8,S1:/\nPETcon-antigen-display-myc,"(root, R1, R3)",downright-ext,QuadrantGate,quadgate_ha_myc,R3,57,0.114,0.250924,3
5,S1:/\nPETcon-antigen-display-myc,"(root, R1, R2)",topleft,QuadrantGate,quadgate,R2,16,0.032,0.065619,3
9,S1:/\nPETcon-antigen-display-myc,"(root, R1, R3)",topleft-ext,QuadrantGate,quadgate_ha_myc,R3,61,0.122,0.268533,3
3,S1:/\nPETcon-antigen-display-myc,"(root, R1, R2)",topright,QuadrantGate,quadgate,R2,5,0.010,0.020506,3


In [46]:
p = session.plot_gate(session.get_sample_ids()[3], 'quadgate',  x_min=0.3, x_max=1, y_min=0.2, y_max=1)
p.width = 300
p.height = 300
show(p)

In [47]:
p = session.plot_gate(session.get_sample_ids()[3], 'quadgate_ha_myc',  x_min=0.3, x_max=1, y_min=0.2, y_max=1)
p.width = 300
p.height = 300
show(p)

# calculate positive

In [48]:
Lratios = []
for sample_id in session.get_sample_ids():
    temp_df = session.get_gating_results(sample_id).report
    topright_ratio = float(temp_df.query('gate_name=="topright"').relative_percent.iloc[0])
    topleft_ratio = float(temp_df.query('gate_name=="topleft"').relative_percent.iloc[0])
    downleft_ratio = float(temp_df.query('gate_name=="downleft"').relative_percent.iloc[0])
    downright_ratio = float(temp_df.query('gate_name=="downright"').relative_percent.iloc[0])

    if downright_ratio > 0:
        positive = round(topright_ratio / (topright_ratio + downright_ratio), 4)
    else:
        positive = 0
    expr = round((topright_ratio + downright_ratio) / 100, 4)
    Lratios.append((sample_id, positive, expr))

In [49]:
df_ratio = pd.DataFrame(Lratios)

In [50]:
df_ratio.columns = ['sample_id', 'positive_ratio', 'expr_ratio']

In [51]:
df.shape

(5, 8)

In [52]:
df = df.join(pl.DataFrame(df_ratio), on='sample_id', how='left').sort('ab')

In [53]:
df.shape

(5, 10)

In [54]:
df.write_csv('FACS_stat.csv')

# batch plot

In [55]:
from bokeh.layouts import row, column, gridplot
from bokeh.plotting import figure
from bokeh.models import Label
from bokeh.io import export_svgs

In [56]:
def plot_grid(sample_ids, cols_per_row=5, title_size='0.8em'):
    plot_list = []
    for smp in sample_ids:
        topright_percent = session.get_gating_results(smp).report.query("gate_name == 'topright'")['relative_percent'].iat[0]
        topright_percent = f'{topright_percent:.2f}%'
        downright_percent = session.get_gating_results(smp).report.query("gate_name == 'downright'")['relative_percent'].iat[0]
        downright_percent = f'{downright_percent:.2f}%'
        downleft_percent = session.get_gating_results(smp).report.query("gate_name == 'downleft'")['relative_percent'].iat[0]
        downleft_percent = f'{downleft_percent:.2f}%'
    
        p = session.plot_gate(smp, 'quadgate', subsample_count=10000,
                                 x_min=0.3, x_max=1, y_min=0.2, y_max=1)
        p.width = 300
        p.height = 300
        p.add_layout(Label(x=0.8, y=0.9, text=topright_percent, text_color="Red"))
        p.add_layout(Label(x=0.8, y=0.4, text=downright_percent, text_color="Red"))
        p.add_layout(Label(x=0.4, y=0.4, text=downleft_percent, text_color="Red"))
        p.xaxis.axis_label = "Myc-FITC"
        p.yaxis.axis_label = "Antibody-PE"
        p.above[1].text_font_size = title_size
        
        plot_list.append(p)
    
    # 将 plot_list 按照 cols_per_row 分组
    grid = [plot_list[i:i + cols_per_row] for i in range(0, len(plot_list), cols_per_row)]
    
    return gridplot(grid)

In [57]:
def plot_grid_ha_myc(sample_ids, cols_per_row=5, title_size='0.8em'):
    plot_list = []
    for smp in sample_ids:
        topright_percent = session.get_gating_results(smp).report.query("gate_name == 'topright-ext'")['relative_percent'].iat[0]
        topright_percent = f'{topright_percent:.2f}%'
        downright_percent = session.get_gating_results(smp).report.query("gate_name == 'downright-ext'")['relative_percent'].iat[0]
        downright_percent = f'{downright_percent:.2f}%'
        downleft_percent = session.get_gating_results(smp).report.query("gate_name == 'downleft-ext'")['relative_percent'].iat[0]
        downleft_percent = f'{downleft_percent:.2f}%'
    
        p = session.plot_gate(smp, 'quadgate_ha_myc', subsample_count=10000,
                                 x_min=0.3, x_max=1, y_min=0.2, y_max=1)
        p.width = 300
        p.height = 300
        p.add_layout(Label(x=0.8, y=0.9, text=topright_percent, text_color="Red"))
        p.add_layout(Label(x=0.8, y=0.4, text=downright_percent, text_color="Red"))
        p.add_layout(Label(x=0.4, y=0.4, text=downleft_percent, text_color="Red"))
        p.xaxis.axis_label = "Myc-FITC"
        p.yaxis.axis_label = "HA-APC"
        p.above[1].text_font_size = title_size
        
        plot_list.append(p)
    
    # 将 plot_list 按照 cols_per_row 分组
    grid = [plot_list[i:i + cols_per_row] for i in range(0, len(plot_list), cols_per_row)]
    
    return gridplot(grid)

In [58]:
df.group_by('cell').len()

cell,len
str,u32
"""PETcon-antigen-display-myc""",5


In [59]:
query_str = 'PETcon-antigen-display-myc'
grid = df.filter(col('cell') == lit(query_str)).sort("ab", "sample")['sample_id'].to_list()
save(plot_grid(grid), f'{batch}-{query_str}.html')
save(plot_grid_ha_myc(grid), f'{batch}-{query_str}-HA-Myc.html')

/tmp/ipykernel_2234992/3224034475.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(plot_grid(grid), f'{batch}-{query_str}.html')
/tmp/ipykernel_2234992/3224034475.py:3: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(plot_grid(grid), f'{batch}-{query_str}.html')
/tmp/ipykernel_2234992/3224034475.py:4: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(plot_grid_ha_myc(grid), f'{batch}-{query_str}-HA-Myc.html')
/tmp/ipykernel_2234992/3224034475.py:4: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(plot_grid_ha_myc(grid), f'{batch}-{query_str}-HA-Myc.html')


'/home/william/jupyter/000-FACS/facs-example/20250908-PETcon-antigen-display-myc-HA-Myc.html'